In this notebook I'll reconstruct the process and results of the DTCR algorithm presented in the "Learning Representations for Time Series Clustering".

In [1]:
# Imports
import torch
from torch.utils.data import DataLoader
from Utilities.DTCR import DTCRModel, DTCRConfig
from Utilities.UCRParser import read_dataset
from Utilities.DRNN import BidirectionalDRNN

In [2]:
train_ds, test_ds = read_dataset("Two_Patterns")
train_dl = DataLoader(train_ds, batch_size=100, shuffle=True)
test_dl = DataLoader(test_ds, batch_size=2, shuffle=True)

config = DTCRConfig()
config.class_num = train_ds.number_of_labels
config.input_size = train_ds[0][0].shape[1]
config.num_steps = train_ds[0][0].shape[0]
config.batch_size = 2


Loading the Two_Patterns dataset...
The dataset Two_Patterns was loaded.


In [3]:
dtcr_model = DTCRModel(config)
dtcr_model

In [6]:
loss = torch.nn.MSELoss()
opt = torch.optim.Adam(list(dtcr_model.parameters()), eps=5e-3)
print_interval = 5
for epoch in range(2):
    running_loss = 0.0
    for index, (sample_data, sample_label) in enumerate(train_dl):
        opt.zero_grad()

        inputs, latent_representation, reconstructed_inputs = dtcr_model(sample_data)
        output = loss(reconstructed_inputs, sample_data)
        output.backward()
        opt.step()

        running_loss += output.item()
        if index % print_interval == print_interval - 1:
            print('[%d, %5d] loss: %.3f' %
                (epoch + 1, index + 1, running_loss / print_interval))
            print("Real values:")
            print(",".join(["{:.2f}".format(float(item)) for item in inputs[0]][:15]))
            print("Latent representation:")
            print(",".join(["{:.2f}".format(float(item)) for item in latent_representation[0]][:15]))
            print("Predicted:")
            print(",".join(["{:.2f}".format(float(item)) for item in reconstructed_inputs[0]][:15]))
            
            running_loss = 0.0


[1,     5] loss: 0.998
Real values:
-0.12,-0.07,0.03,0.20,-0.67,-0.22,0.61,-0.28,0.16,0.32,0.27,0.22,-0.05,-0.34,0.36
Latent representation:
-0.03,0.05,0.14,-0.09,0.15,-0.00,-0.02,0.00,0.05,-0.03,0.03,0.06,-0.03,0.07,0.07
Predicted:
-0.07,-0.04,0.01,0.04,0.05,0.05,0.05,0.04,0.03,0.03,0.03,0.03,0.02,0.02,0.03
[1,    10] loss: 0.993
Real values:
0.04,-0.25,-0.04,-0.87,-0.45,0.40,0.55,-0.36,0.19,0.31,-0.60,-0.35,0.51,0.38,0.26
Latent representation:
-0.02,0.07,0.13,-0.07,0.14,-0.03,-0.05,0.02,0.05,-0.02,0.04,0.06,-0.02,0.08,0.03
Predicted:
-0.01,-0.02,-0.02,-0.01,-0.00,-0.01,-0.01,-0.01,-0.02,-0.02,-0.02,-0.03,-0.03,-0.03,-0.03
[2,     5] loss: 0.992
Real values:
0.32,0.21,-0.41,0.16,0.22,0.21,-0.42,-0.37,0.12,-0.97,-0.02,-0.00,-0.45,0.09,-0.18
Latent representation:
-0.00,0.08,0.13,-0.05,0.12,-0.04,-0.07,0.02,0.04,-0.01,0.04,0.08,-0.00,0.08,0.01
Predicted:
0.08,0.05,0.04,0.03,0.03,0.02,0.02,0.02,0.01,0.01,0.01,0.01,0.01,0.01,0.01
[2,    10] loss: 0.991
Real values:
-0.64,-0.14,-0.25,-0.3